<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study2/blob/main/apple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import os
import time  # time 모듈 import
import psutil  # psutil 라이브러리 import
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical
import keras.backend as K

In [15]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_data'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

In [16]:
# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

In [17]:
# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [18]:
# CNN 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(categories), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# 모델 학습
history = model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_val, y_val_categorical))

Epoch 1/20
219/219 [==============================] - 78s 351ms/step - loss: 1.6762 - accuracy: 0.4827 - val_loss: 1.0807 - val_accuracy: 0.5869
Epoch 2/20
219/219 [==============================] - 76s 346ms/step - loss: 0.8074 - accuracy: 0.6503 - val_loss: 0.3419 - val_accuracy: 0.8623
Epoch 3/20
219/219 [==============================] - 76s 347ms/step - loss: 0.4435 - accuracy: 0.8094 - val_loss: 0.2801 - val_accuracy: 0.8654
Epoch 4/20
219/219 [==============================] - 76s 347ms/step - loss: 0.3372 - accuracy: 0.8604 - val_loss: 0.1170 - val_accuracy: 0.9726
Epoch 5/20
219/219 [==============================] - 76s 348ms/step - loss: 0.2504 - accuracy: 0.8972 - val_loss: 0.0638 - val_accuracy: 0.9914
Epoch 6/20
219/219 [==============================] - 76s 349ms/step - loss: 0.2188 - accuracy: 0.9018 - val_loss: 0.0405 - val_accuracy: 1.0000
Epoch 7/20
219/219 [==============================] - 76s 348ms/step - loss: 0.2137 - accuracy: 0.9158 - val_loss: 0.0742 - val_ac

In [20]:
# 모델 사이즈 확인
model.save('model.h5')
model_size = os.path.getsize('model.h5') / (1024 * 1024)  # 모델 사이즈를 MB 단위로 변환
print(f"Model Size: {model_size:.2f} MB")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model Size: 439.50 MB


In [21]:
# 메모리 사용량 확인
process = psutil.Process(os.getpid())
memory_usage = process.memory_info().rss / (1024 * 1024)  # 메모리 사용량을 MB 단위로 변환
print(f"Memory Usage: {memory_usage:.2f} MB")

Memory Usage: 7954.64 MB


In [22]:
# 추론 시간 측정
start_time = time.time()
y_pred_categorical = model.predict(X_test)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.2f} seconds")

y_pred = np.argmax(y_pred_categorical, axis=1)

40/40 [==============================] - 3s 70ms/step
Inference Time: 3.12 seconds


In [23]:
# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val_categorical)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

40/40 [==============================] - 3s 69ms/step - loss: 0.0049 - accuracy: 1.0000
Validation Loss: 0.004938323050737381
Validation Accuracy: 1.0


In [24]:
# 테스트 데이터 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

40/40 [==============================] - 3s 68ms/step - loss: 0.0096 - accuracy: 0.9992
Test Loss: 0.00957194622606039
Test Accuracy: 0.9992175102233887


In [25]:
# 혼동 행렬 및 성능 지표 출력 (검증 데이터)
y_pred_val = model.predict(X_val)
y_pred_val_classes = np.argmax(y_pred_val, axis=1)

conf_matrix_val = confusion_matrix(y_val_encoded, y_pred_val_classes)
class_report_val = classification_report(y_val_encoded, y_pred_val_classes, target_names=categories)

print("Confusion Matrix (Validation):")
print(conf_matrix_val)
print("\nClassification Report (Validation):")
print(class_report_val)

40/40 [==============================] - 3s 67ms/step
Confusion Matrix (Validation):
[[176   0   0   0]
 [  0 176   0   0]
 [  0   0 176   0]
 [  0   0   0 750]]

Classification Report (Validation):
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00       176
    fault_BB       1.00      1.00      1.00       176
    fault_RI       1.00      1.00      1.00       176
    fault_SM       1.00      1.00      1.00       750

    accuracy                           1.00      1278
   macro avg       1.00      1.00      1.00      1278
weighted avg       1.00      1.00      1.00      1278



In [26]:
# 혼동 행렬 및 성능 지표 출력 (테스트 데이터)
conf_matrix_test = confusion_matrix(y_test_encoded, y_pred)
class_report_test = classification_report(y_test_encoded, y_pred, target_names=categories)

print("Confusion Matrix (Test):")
print(conf_matrix_test)
print("\nClassification Report (Test):")
print(class_report_test)

Confusion Matrix (Test):
[[176   0   0   0]
 [  0 176   0   0]
 [  1   0 175   0]
 [  0   0   0 750]]

Classification Report (Test):
              precision    recall  f1-score   support

      normal       0.99      1.00      1.00       176
    fault_BB       1.00      1.00      1.00       176
    fault_RI       1.00      0.99      1.00       176
    fault_SM       1.00      1.00      1.00       750

    accuracy                           1.00      1278
   macro avg       1.00      1.00      1.00      1278
weighted avg       1.00      1.00      1.00      1278



In [ ]:
# 새로운 CSV 파일 분류
def classify_new_file(file_path, model):
    data = pd.read_csv(file_path, header=None).values
    data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
    data = np.expand_dims(data, axis=(0, -1))
    prediction = model.predict(data)
    predicted_class = np.argmax(prediction, axis=1)
    return label_encoder.inverse_transform(predicted_class)

# 예시: 새로운 파일 분류
new_file_path = 'path_to_new_csv_file.csv'
predicted_category = classify_new_file(new_file_path, model)
print(f'The predicted category for the new file is: {predicted_category[0]}')